In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
import unidecode

import folium
from folium import plugins

import geopy
from geopy.geocoders import Nominatim
#geolocator = Nominatim(user_agent="https://nominatim.openstreetmap.org/")
geolocator = Nominatim(user_agent="https://maps.googleapis.com/")

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()


pd.options.plotting.backend = "plotly"
pd.set_option('max_columns', None)

In [2]:
#reading/storing and showing the Covid dataset of Colombia
#df=pd.read_csv('Casos_positivos_de_COVID-19_en_Colombia.csv')
#df=pd.read_csv('Covid18-09-20.csv')
#df.to_pickle('Covid18-09-20.plk')

df=pd.read_pickle('Covid18-09-20.plk')
df.head()

,ID de caso,Fecha de notificación,Código DIVIPOLA,Ciudad de ubicación,Departamento o Distrito,atención,Edad,Sexo,Tipo,Estado,País de procedencia,FIS,Fecha de muerte,Fecha diagnostico,Fecha recuperado,fecha reporte web,Tipo recuperación,Codigo departamento,Codigo pais,Pertenencia etnica,Nombre grupo etnico
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,ITALIA,2020-02-27T00:00:00.000,NaN,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR,11,380.0,Otro,NaN
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,ESPAÑA,2020-03-04T00:00:00.000,NaN,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR,76,724.0,Negro,NaN
2,3,2020-03-07T00:00:00.000,5001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,ESPAÑA,2020-02-29T00:00:00.000,NaN,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,PCR,5,724.0,Otro,NaN
3,4,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,NaN,2020-03-06T00:00:00.000,NaN,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN
4,5,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,NaN,2020-03-08T00:00:00.000,NaN,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN


In [3]:
#show the features and the len of the data
df.shape

(743945, 21)

We have 22 different geatures and 357710 of data

In [4]:
#show the all the features (columns)
df.columns

Index(['ID de caso', 'Fecha de notificación', 'Código DIVIPOLA',
       'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad',
       'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS',
       'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado',
       'fecha reporte web', 'Tipo recuperación', 'Codigo departamento',
       'Codigo pais', 'Pertenencia etnica', 'Nombre grupo etnico'],
      dtype='object')

Formating Columns headders

In [5]:
df.columns = df.columns.str.upper()
#df.columns = map(str.upper, df.columns)
df.head(2)

,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,ITALIA,2020-02-27T00:00:00.000,NaN,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR,11,380.0,Otro,NaN
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,ESPAÑA,2020-03-04T00:00:00.000,NaN,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR,76,724.0,Negro,NaN


In [6]:
# info() show a description indicating the values and type of each feature
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743945 entries, 0 to 743944
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID DE CASO                743945 non-null  int64  
 1   FECHA DE NOTIFICACIÓN     743945 non-null  object 
 2   CÓDIGO DIVIPOLA           743945 non-null  int64  
 3   CIUDAD DE UBICACIÓN       743945 non-null  object 
 4   DEPARTAMENTO O DISTRITO   743945 non-null  object 
 5   ATENCIÓN                  742389 non-null  object 
 6   EDAD                      743945 non-null  int64  
 7   SEXO                      743945 non-null  object 
 8   TIPO                      743945 non-null  object 
 9   ESTADO                    742175 non-null  object 
 10  PAÍS DE PROCEDENCIA       968 non-null     object 
 11  FIS                       651137 non-null  object 
 12  FECHA DE MUERTE           25317 non-null   object 
 13  FECHA DIAGNOSTICO         739876 non-null  o

there are some features with missing values and erratic dtypes, in order to fix that we need to look each of them.

In [7]:
# Looking the structure of the date features 

col_de_fechas=[col for col in df if col.startswith('F')]

df[col_de_fechas].head()

,FECHA DE NOTIFICACIÓN,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB
0,2020-03-02T00:00:00.000,2020-02-27T00:00:00.000,NaN,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000
1,2020-03-06T00:00:00.000,2020-03-04T00:00:00.000,NaN,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000
2,2020-03-07T00:00:00.000,2020-02-29T00:00:00.000,NaN,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000
3,2020-03-09T00:00:00.000,2020-03-06T00:00:00.000,NaN,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000
4,2020-03-09T00:00:00.000,2020-03-08T00:00:00.000,NaN,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000


Notice that each date feature have a T between the date and the time, so we need to remove the T and remove the time, as we only have 00:00:00:000

In [8]:
# the following code remove the T from the dates features but also store the time in the unwanted column
for fechas in col_de_fechas:
    df[[fechas,'unwanted']]=df[fechas].str.split(pat="T",expand=True)

In [9]:
df[col_de_fechas].head()

,FECHA DE NOTIFICACIÓN,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB
0,2020-03-02,2020-02-27,NaN,2020-03-06,2020-03-13,2020-03-06
1,2020-03-06,2020-03-04,NaN,2020-03-09,2020-03-19,2020-03-09
2,2020-03-07,2020-02-29,NaN,2020-03-09,2020-03-15,2020-03-09
3,2020-03-09,2020-03-06,NaN,2020-03-11,2020-03-26,2020-03-11
4,2020-03-09,2020-03-08,NaN,2020-03-11,2020-03-23,2020-03-11


In [10]:
df['unwanted'].unique()

array(['00:00:00.000'], dtype=object)

As unwanted does provide a useful time, we will get rid of that feature

In [11]:
#drop the column unwanted and save the changes to the df (inplace=True)
df.drop(columns=['unwanted'],inplace=True)

Now we transform the dtype form objecto to a datetime format

In [12]:
for col in col_de_fechas:
    df[col]=df[col].apply(lambda x: pd.NaT if x=='ERROR: #N/A' else x)

In [13]:
#Create a new column called Asintomático FIS to store  the values of 'Asintomático' from FIS
df['ASINTOMÁTICO FIS']=df['FIS'].progress_apply(lambda x: 'Asintomático' if x=='Asintomático' else np.nan)
#Removing 'Asintomático' from FIS columns and convert to datetime
df.FIS=df['FIS'].progress_apply(lambda x:  pd.NaT if x=='Asintomático' else pd.to_datetime(x,format='%Y-%m-%d'))

In [14]:
#import re
#if re.match("^[a-zA-Z]+.*", line):

In [15]:
#fechas_to_date=['Fecha de notificación','Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado','fecha reporte web']

for fechas in col_de_fechas:
    df[fechas]= pd.to_datetime(df[fechas],format='%Y-%m-%d')

In [16]:
df.select_dtypes(include='datetime').dtypes

FECHA DE NOTIFICACIÓN    datetime64[ns]
FIS                      datetime64[ns]
FECHA DE MUERTE          datetime64[ns]
FECHA DIAGNOSTICO        datetime64[ns]
FECHA RECUPERADO         datetime64[ns]
FECHA REPORTE WEB        datetime64[ns]
dtype: object

In [17]:
print(df.ESTADO.unique())
print(df.TIPO.unique())
print(df.ATENCIÓN.unique())

['Leve' 'Asintomático' 'Fallecido' nan 'Moderado' 'Grave']
['Importado' 'Relacionado' 'En estudio' 'relacionado' 'RELACIONADO'
 'En Estudio']
['Recuperado' 'Fallecido' nan 'Casa' 'Hospital' 'Hospital UCI']


Notice that some feature have duplicated values, so we goin to see me unique values for certain features

In [18]:
for columna in df.select_dtypes(exclude='datetime').columns:#.drop(['ID de caso','Código DIVIPOLA','Ciudad de ubicación'],axis=1).columns:
    print('------------------------------------------')
    print(columna)
    print(len(df[columna].unique()))

------------------------------------------
ID DE CASO
743945
------------------------------------------
CÓDIGO DIVIPOLA
1042
------------------------------------------
CIUDAD DE UBICACIÓN
967
------------------------------------------
DEPARTAMENTO O DISTRITO 
37
------------------------------------------
ATENCIÓN
6
------------------------------------------
EDAD
112
------------------------------------------
SEXO
4
------------------------------------------
TIPO
6
------------------------------------------
ESTADO
6
------------------------------------------
PAÍS DE PROCEDENCIA
51
------------------------------------------
TIPO RECUPERACIÓN
3
------------------------------------------
CODIGO DEPARTAMENTO
33
------------------------------------------
CODIGO PAIS
46
------------------------------------------
PERTENENCIA ETNICA
5
------------------------------------------
NOMBRE GRUPO ETNICO
744
------------------------------------------
ASINTOMÁTICO FIS
1


In order to fix duplicate values were are goin to  remove punctuation and only leave uppercase strings.
For numeric values is a bit more difficult as we have codes for municipalities, but we can see the Edad (Age) feature.

---------

In [19]:
#example of the strip function for str values
txt = ",,,!!!,,r--rttgg...   ..bananá....__rrr"
x = txt.strip(",.grt! -_")
print(x)

bananá


In [20]:
# Example of how to remove accent
accented_string = 'Málaga'
# accented_string is of type 'unicode'
import unidecode
unidecode.unidecode(accented_string)
# unaccented_string contains 'Malaga'and is of type 'str'

'Malaga'

In [21]:
unidecode.unidecode(accented_string.upper())

'MALAGA'

-------

For categorical variables we'll change nan values to a 'No info' string

In [22]:
# Filling nan values, applying uppes case and remove accent for each value of each feature
for feature in tqdm(df.select_dtypes(include=['object']).columns):
    df[feature]=df[feature].fillna('No info')
    df[feature]=df[feature].apply(lambda x: unidecode.unidecode(x.upper()))
    print('Compleated feature: '+feature)

100%|██████████| 10/10 [02:43<00:00, 16.32s/it]Compleated feature: NOMBRE GRUPO ETNICO



In [23]:
df.select_dtypes(include=['object']).head()

,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,TIPO RECUPERACIÓN,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,F,IMPORTADO,LEVE,ITALIA,PCR,OTRO,NO INFO
1,GUADALAJARA DE BUGA,VALLE DEL CAUCA,RECUPERADO,M,IMPORTADO,LEVE,ESPANA,PCR,NEGRO,NO INFO
2,MEDELLIN,ANTIOQUIA,RECUPERADO,F,IMPORTADO,LEVE,ESPANA,PCR,OTRO,NO INFO
3,MEDELLIN,ANTIOQUIA,RECUPERADO,M,RELACIONADO,LEVE,NO INFO,PCR,OTRO,NO INFO
4,MEDELLIN,ANTIOQUIA,RECUPERADO,M,RELACIONADO,LEVE,NO INFO,PCR,OTRO,NO INFO


In [24]:
for columna in df.select_dtypes(exclude='datetime').columns:#.drop(['ID de caso','Código DIVIPOLA','Ciudad de ubicación'],axis=1).columns:
    print('------------------------------------------')
    print(columna)
    print(len(df[columna].unique()))

------------------------------------------
ID DE CASO
743945
------------------------------------------
CÓDIGO DIVIPOLA
1042
------------------------------------------
CIUDAD DE UBICACIÓN
966
------------------------------------------
DEPARTAMENTO O DISTRITO 
37
------------------------------------------
ATENCIÓN
6
------------------------------------------
EDAD
112
------------------------------------------
SEXO
2
------------------------------------------
TIPO
3
------------------------------------------
ESTADO
6
------------------------------------------
PAÍS DE PROCEDENCIA
48
------------------------------------------
TIPO RECUPERACIÓN
3
------------------------------------------
CODIGO DEPARTAMENTO
33
------------------------------------------
CODIGO PAIS
46
------------------------------------------
PERTENENCIA ETNICA
5
------------------------------------------
NOMBRE GRUPO ETNICO
716
------------------------------------------
ASINTOMÁTICO FIS
1


In [25]:
# same code as before but using list mprehension and converting it into a DataFrame
pd.DataFrame([[p,len(df[p].unique())] for p in df.select_dtypes(exclude='datetime').columns],columns=['feature','len'])

,feature,len
0,ID DE CASO,743945
1,CÓDIGO DIVIPOLA,1042
2,CIUDAD DE UBICACIÓN,966
3,DEPARTAMENTO O DISTRITO,37
4,ATENCIÓN,6
5,EDAD,112
6,SEXO,2
7,TIPO,3
8,ESTADO,6
9,PAÍS DE PROCEDENCIA,48


We saw that some of the values of the features reduce, bc duplicated values has been removed.


In [26]:
for columna in df.select_dtypes(exclude='datetime').drop(['ID DE CASO','CÓDIGO DIVIPOLA','CIUDAD DE UBICACIÓN'],axis=1).columns:
    if len(df[columna].unique())<10:
        print('------------------------------------------')
        print(columna)
        print(df[columna].unique())

------------------------------------------
ATENCIÓN
['RECUPERADO' 'FALLECIDO' 'NO INFO' 'CASA' 'HOSPITAL' 'HOSPITAL UCI']
------------------------------------------
SEXO
['F' 'M']
------------------------------------------
TIPO
['IMPORTADO' 'RELACIONADO' 'EN ESTUDIO']
------------------------------------------
ESTADO
['LEVE' 'ASINTOMATICO' 'FALLECIDO' 'NO INFO' 'MODERADO' 'GRAVE']
------------------------------------------
TIPO RECUPERACIÓN
['PCR' 'TIEMPO' 'NO INFO']
------------------------------------------
PERTENENCIA ETNICA
['OTRO' 'NEGRO' 'INDIGENA' 'ROM' 'NO INFO']
------------------------------------------
ASINTOMÁTICO FIS
[nan]


Next thing to do is check if the ```atención values``` agrees with ```Estado```, and dates match , i.g.  we don't want  that feature 'Estado'='Fallecido' have a different value in the feature 'atención'

In [27]:
for atenticion in df['ATENCIÓN'].unique():
    print('------------------------')
    print('ATENCIÓN: '+atenticion)
    print(df[df['ATENCIÓN']==atenticion]['ESTADO'].value_counts())

------------------------
ATENCIÓN: RECUPERADO
LEVE            537128
ASINTOMATICO     67653
MODERADO          9221
GRAVE             1162
NO INFO            293
Name: ESTADO, dtype: int64
------------------------
ATENCIÓN: FALLECIDO
FALLECIDO    23665
Name: ESTADO, dtype: int64
------------------------
ATENCIÓN: NO INFO
NO INFO         1477
ASINTOMATICO      79
Name: ESTADO, dtype: int64
------------------------
ATENCIÓN: CASA
LEVE            63939
ASINTOMATICO    24684
Name: ESTADO, dtype: int64
------------------------
ATENCIÓN: HOSPITAL
MODERADO        13218
ASINTOMATICO      284
Name: ESTADO, dtype: int64
------------------------
ATENCIÓN: HOSPITAL UCI
GRAVE           1034
ASINTOMATICO     108
Name: ESTADO, dtype: int64


Remark for the unique values of  the feature Estado:

* 'Recuperado' have LEVE, MODERADO, GRAVE, ASINTOMATICO and NO INFO. maybe that was the health condition of those  people.

* 'Fallecido' seems good has it have no different values despite FALLECIIDO

* 'HOSPITAL' only have MODERADO and ASINTOMATICO, for that las values his wird to have it in HOSPITAL.

* 'HOSPITAL UCI' GRAVE seems good for that health condition BUT IT IS WEIRD TO THINK THAT ASINTOMATICO IS IN UCI.

* 'NO INFO'  contains NO INFO and ASINTOMATICO

* 'CASA' have LEVE and ASINTOMATICO, wich seems good.

We need to drop  Estado= ASINTOMATICO where atención= HOSPITAL and atención=HOSPITAL UCI

In [28]:
#getting the index 
idx_h=df[(df['ESTADO']=='ASINTOMATICO') & (df['ATENCIÓN']=='HOSPITAL')].index
idx_hu=df[(df['ESTADO']=='ASINTOMATICO') & (df['ATENCIÓN']=='HOSPITAL UCI')].index

#printing the shape of the df when droping those indexes 
print(df.shape)
print(df.drop(idx_h).shape)
print(df.drop(idx_hu).shape)

(743945, 22)
(743661, 22)
(743837, 22)


In [29]:
#droping those indexes in the data frame
df.drop(idx_h,inplace=True)
df.drop(idx_hu,inplace=True)

In [30]:
for atenticion in df['ATENCIÓN'].unique():
    print('------------------------')
    print('ATENCIÓN: '+atenticion)
    print(df[df['ATENCIÓN']==atenticion]['TIPO RECUPERACIÓN'].value_counts())


------------------------
ATENCIÓN: RECUPERADO
TIEMPO    510302
PCR       105155
Name: TIPO RECUPERACIÓN, dtype: int64
------------------------
ATENCIÓN: FALLECIDO
NO INFO    23665
Name: TIPO RECUPERACIÓN, dtype: int64
------------------------
ATENCIÓN: NO INFO
NO INFO    1556
Name: TIPO RECUPERACIÓN, dtype: int64
------------------------
ATENCIÓN: CASA
NO INFO    88623
Name: TIPO RECUPERACIÓN, dtype: int64
------------------------
ATENCIÓN: HOSPITAL
NO INFO    13218
Name: TIPO RECUPERACIÓN, dtype: int64
------------------------
ATENCIÓN: HOSPITAL UCI
NO INFO    1034
Name: TIPO RECUPERACIÓN, dtype: int64


Seems good for every feature, becasue NO INFO means there is not values (nan). And RECUPERADO must be the only one with valid data.

Check if ASINTOMATICO FIS have different values for Estado when  Estado='ASINTOMATICO'

In [31]:
#si tipo de recuperacón presente solo esta para los de atención=RECUPERADO
df[~(df['ASINTOMÁTICO FIS']=='ASINTOMATICO')]['ESTADO'].unique()

array(['LEVE', 'ASINTOMATICO', 'FALLECIDO', 'NO INFO', 'MODERADO',
       'GRAVE'], dtype=object)

ASINTOMÁTICO FIS=ASINTOMATICO must not appear when looking in Estado!= ASINTOMATICO.

In [32]:
t=df[~(df['ASINTOMÁTICO FIS']=='ASINTOMATICO')]
t[t.ESTADO=='ASINTOMATICO']

,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO,ASINTOMÁTICO FIS
25,26,2020-03-15,13001,CARTAGENA DE INDIAS,CARTAGENA D.T. Y C.,RECUPERADO,30,F,RELACIONADO,ASINTOMATICO,NO INFO,NaT,NaT,2020-03-15,2020-03-25,2020-03-15,PCR,13,NaN,OTRO,NO INFO,NaN
26,27,2020-03-15,13001,CARTAGENA DE INDIAS,CARTAGENA D.T. Y C.,RECUPERADO,30,F,RELACIONADO,ASINTOMATICO,NO INFO,NaT,NaT,2020-03-15,2020-03-25,2020-03-15,PCR,13,NaN,OTRO,NO INFO,NaN
40,41,2020-03-14,41001,NEIVA,HUILA,RECUPERADO,79,M,RELACIONADO,ASINTOMATICO,NO INFO,NaT,NaT,2020-03-14,2020-03-28,2020-03-14,PCR,41,NaN,OTRO,NO INFO,NaN
47,48,2020-03-15,11001,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,31,M,RELACIONADO,ASINTOMATICO,NO INFO,NaT,NaT,2020-03-16,2020-03-27,2020-03-16,PCR,11,NaN,OTRO,NO INFO,NaN
84,85,2020-03-17,76001,CALI,VALLE DEL CAUCA,RECUPERADO,42,F,RELACIONADO,ASINTOMATICO,NO INFO,NaT,NaT,2020-03-17,2020-05-31,2020-03-17,PCR,76,NaN,OTRO,NO INFO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742122,742163,2020-09-15,11001,BOGOTA D.C.,BOGOTA D.C.,CASA,62,M,EN ESTUDIO,ASINTOMATICO,NO INFO,NaT,NaT,2020-09-15,NaT,2020-09-17,NO INFO,11,NaN,NO INFO,NO INFO,NaN
742123,742164,2020-09-16,11001,BOGOTA D.C.,BOGOTA D.C.,CASA,29,M,EN ESTUDIO,ASINTOMATICO,NO INFO,NaT,NaT,2020-09-16,NaT,2020-09-17,NO INFO,11,NaN,NO INFO,NO INFO,NaN
742309,742350,2020-09-13,54001,CUCUTA,NORTE DE SANTANDER,CASA,2,F,EN ESTUDIO,ASINTOMATICO,NO INFO,NaT,NaT,2020-09-16,NaT,2020-09-17,NO INFO,54,NaN,NO INFO,NO INFO,NaN
742334,742375,2020-09-14,8001,BARRANQUILLA,BARRANQUILLA D.E.,CASA,50,F,EN ESTUDIO,ASINTOMATICO,NO INFO,NaT,NaT,2020-09-15,NaT,2020-09-17,NO INFO,8,NaN,NO INFO,NO INFO,NaN


In [33]:
df[(df['ESTADO']=='ASINTOMATICO')]['ASINTOMÁTICO FIS'].unique()

array([nan])

Seem that FIS has been fixed, as in previous versions of the database,FIS contains string values as 'Asintomático'

In [34]:
#dropping 'ASINTOMÁTICO FIS'
df.drop(columns=['ASINTOMÁTICO FIS'], inplace=True)

Checking the 'Tipo' values for each unique value of 'atención'

In [35]:
for values in df['ATENCIÓN'].unique():
    print('-------------------------')
    print(values)
    print(df[df['ATENCIÓN']==values]['TIPO'].value_counts())

-------------------------
RECUPERADO
EN ESTUDIO     577183
RELACIONADO     37328
IMPORTADO         946
Name: TIPO, dtype: int64
-------------------------
FALLECIDO
EN ESTUDIO     23006
RELACIONADO      636
IMPORTADO         23
Name: TIPO, dtype: int64
-------------------------
NO INFO
EN ESTUDIO     1513
RELACIONADO      43
Name: TIPO, dtype: int64
-------------------------
CASA
EN ESTUDIO     87819
RELACIONADO      800
IMPORTADO          4
Name: TIPO, dtype: int64
-------------------------
HOSPITAL
EN ESTUDIO     13110
RELACIONADO      104
IMPORTADO          4
Name: TIPO, dtype: int64
-------------------------
HOSPITAL UCI
EN ESTUDIO     1026
RELACIONADO       8
Name: TIPO, dtype: int64


Validating that we have not different values for atentción feature when Estado = 'FALLECIDO'

In [36]:
df[(df['ESTADO']=='FALLECIDO') & (df['ATENCIÓN']!='FALLECIDO')]

,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO


Seem's good.

Next thing to do is check that we have no dates like Fecha de muerte prior fecha de notificación or diagnostico

In [37]:
df.select_dtypes(include='datetime').isnull().any()

FECHA DE NOTIFICACIÓN    False
FIS                       True
FECHA DE MUERTE           True
FECHA DIAGNOSTICO         True
FECHA RECUPERADO          True
FECHA REPORTE WEB        False
dtype: bool

In [38]:
df.select_dtypes(include='datetime').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743553 entries, 0 to 743944
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   FECHA DE NOTIFICACIÓN  743553 non-null  datetime64[ns]
 1   FIS                    651137 non-null  datetime64[ns]
 2   FECHA DE MUERTE        25290 non-null   datetime64[ns]
 3   FECHA DIAGNOSTICO      739484 non-null  datetime64[ns]
 4   FECHA RECUPERADO       615457 non-null  datetime64[ns]
 5   FECHA REPORTE WEB      743553 non-null  datetime64[ns]
dtypes: datetime64[ns](6)
memory usage: 39.7 MB


We have various dates with null values, need to take in mind those when cheking for prior dates

Update the df, using only the index that met the Fecha de notificación <= Fecha reporte web

In [39]:
s=(df['FECHA REPORTE WEB']-df['FECHA DE NOTIFICACIÓN'])
print((s<dt.timedelta(0)).value_counts())
#Updating the df with the values that met the condition
df=df.loc[s[(s>=dt.timedelta(0))].index]
print(df.shape)
df.head()

False    743069
True        484
dtype: int64
(743069, 21)


,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02,11001,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,19,F,IMPORTADO,LEVE,ITALIA,2020-02-27,NaT,2020-03-06,2020-03-13,2020-03-06,PCR,11,380.0,OTRO,NO INFO
1,2,2020-03-06,76111,GUADALAJARA DE BUGA,VALLE DEL CAUCA,RECUPERADO,34,M,IMPORTADO,LEVE,ESPANA,2020-03-04,NaT,2020-03-09,2020-03-19,2020-03-09,PCR,76,724.0,NEGRO,NO INFO
2,3,2020-03-07,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,50,F,IMPORTADO,LEVE,ESPANA,2020-02-29,NaT,2020-03-09,2020-03-15,2020-03-09,PCR,5,724.0,OTRO,NO INFO
3,4,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,55,M,RELACIONADO,LEVE,NO INFO,2020-03-06,NaT,2020-03-11,2020-03-26,2020-03-11,PCR,5,NaN,OTRO,NO INFO
4,5,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,25,M,RELACIONADO,LEVE,NO INFO,2020-03-08,NaT,2020-03-11,2020-03-23,2020-03-11,PCR,5,NaN,OTRO,NO INFO


We have 205 values where Fecha reporte web  is prior to fecha de notificación.

Next the following condution must be met:

* 'FIS'<'Fecha diagnostico'<'Fecha recuperado'

* 'FIS'<'Fecha diagnostico'<'Fecha de muerte'

In [40]:
# # Bad approach as we comparing items with missing values
# s2=(df['FIS']<df['Fecha diagnostico'])
# print(s2.value_counts())
# s2[~s2].index

In [41]:
# Because we dont have every value, we just focus in those who are NaT. Compare only if both series have a value. otherwise skip
idx_met=[]
idx_nmet=[]
idx_null=[]
for x in tqdm(df.index):
    if pd.notna(df['FIS'][x]) and pd.notna(df['FECHA DIAGNOSTICO'][x]):
        if df['FIS'][x]<df['FECHA DIAGNOSTICO'][x]:
            idx_met.append(x)
        else:
            idx_nmet.append(x)
    else:
        idx_null.append(x)

100%|██████████| 743069/743069 [21:57<00:00, 563.96it/s]


### Checking why we ahve NaT values in FECHA DIAGNOSTICO 

In [42]:
df[df['FECHA DIAGNOSTICO']<df['FECHA DE NOTIFICACIÓN']][['FECHA DIAGNOSTICO','FECHA DE NOTIFICACIÓN','FIS']]

,FECHA DIAGNOSTICO,FECHA DE NOTIFICACIÓN,FIS
4778,2020-03-23,2020-03-24,2020-03-15
6604,2020-04-30,2020-05-01,NaT
6605,2020-04-30,2020-05-01,NaT
6606,2020-04-30,2020-05-01,NaT
6607,2020-04-30,2020-05-01,NaT
...,...,...,...
731107,2020-09-03,2020-09-15,2020-08-20
731145,2020-08-28,2020-09-15,2020-08-16
731184,2020-08-25,2020-09-10,2020-08-11
731226,2020-08-09,2020-08-27,2020-08-03


the majority of Dates of FECHA DIAGNOSTICO are post FECHA DE NOTIFICACIÓN, so what is the diference between those ?

In [43]:
df[df['FECHA DE NOTIFICACIÓN']<df['FIS']]

,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO


It is a good sign that  we do not have dates FECHA DE NOTIFICACIÓN post FIS.

But what happens insigts privide NaT values of FECHA DIAGNOSTICO?

In [44]:
df[df['FECHA DIAGNOSTICO'].isna()]['ESTADO'].value_counts()

LEVE            3537
FALLECIDO        204
ASINTOMATICO      38
MODERADO          36
NO INFO           27
GRAVE             10
Name: ESTADO, dtype: int64

In [45]:
print('Percentage {:2.3f}% of missing FECHA DIAGNOSTICO dates in regard with the whole data '.format(100*len(df[df['FECHA DIAGNOSTICO'].isna()])/len(df[pd.notnull(df['FECHA DIAGNOSTICO'])])),len(df))

Percentage 0.521% of missing FECHA DIAGNOSTICO dates in regard with the whole data  743069


Two things that we can see is that the percentage of missing dates are not meaningful and we have different values for ESTADO.

In order with that, I considered two actions to take:

1. Drop every NaT value for FECHA DIAGNOSTICO
2. Work with FECHA NOTIFICACÓN  instead of FECHA DIAGNOSTICO.

I choose to the the first action (1.) because it make sense that only people that have been seem by a doctor or related are the ones with/without the virus and the amount of data with out the date is low compared with the whole data.

In [103]:
#Droping NaT row from FECHA DIAGNOSTICO in df
df.drop(df[df['FECHA DIAGNOSTICO'].isna()].index,inplace=True)

In [47]:
df[df['FIS'].isna()]['ESTADO'].value_counts()

ASINTOMATICO    92396
Name: ESTADO, dtype: int64

In [48]:
df[pd.notnull(df['FIS'])]['ESTADO'].value_counts()

LEVE         600653
FALLECIDO     23640
MODERADO      22416
GRAVE          2194
NO INFO        1770
Name: ESTADO, dtype: int64

In [49]:
print(idx_met[-10:])
print('Len of index that met the condition: {}'.format(len(idx_met)))
print('Len of index that NOT met the condition: {}'.format(len(idx_nmet)))
print('Len of Null index : {}'.format(len(idx_null)))

[743935, 743936, 743937, 743938, 743939, 743940, 743941, 743942, 743943, 743944]
Len of index that met the condition: 641738
Len of index that NOT met the condition: 5121
Len of Null index : 96210


We have 1050 that not met the condition 'FIS'<'Fecha diagnostico', So we need to drop those index

In [50]:
#Droping index that not met
df.drop(idx_nmet,inplace=True)
print(df.shape)
df.head()

(737948, 21)


,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02,11001,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,19,F,IMPORTADO,LEVE,ITALIA,2020-02-27,NaT,2020-03-06,2020-03-13,2020-03-06,PCR,11,380.0,OTRO,NO INFO
1,2,2020-03-06,76111,GUADALAJARA DE BUGA,VALLE DEL CAUCA,RECUPERADO,34,M,IMPORTADO,LEVE,ESPANA,2020-03-04,NaT,2020-03-09,2020-03-19,2020-03-09,PCR,76,724.0,NEGRO,NO INFO
2,3,2020-03-07,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,50,F,IMPORTADO,LEVE,ESPANA,2020-02-29,NaT,2020-03-09,2020-03-15,2020-03-09,PCR,5,724.0,OTRO,NO INFO
3,4,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,55,M,RELACIONADO,LEVE,NO INFO,2020-03-06,NaT,2020-03-11,2020-03-26,2020-03-11,PCR,5,NaN,OTRO,NO INFO
4,5,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,25,M,RELACIONADO,LEVE,NO INFO,2020-03-08,NaT,2020-03-11,2020-03-23,2020-03-11,PCR,5,NaN,OTRO,NO INFO


Now we need to check:

* 'Fecha diagnostico'<'Fecha recuperado'

In [51]:
idx_met=[]
idx_nmet=[]
idx_null=[]
for x in tqdm(df.index):
    if pd.notna(df['FECHA DIAGNOSTICO'][x]) and pd.notna(df['FECHA RECUPERADO'][x]):
        if df['FECHA DIAGNOSTICO'][x]<df['FECHA RECUPERADO'][x]:
            idx_met.append(x)
        else:
            idx_nmet.append(x)
    else:
        idx_null.append(x)

100%|██████████| 737948/737948 [05:44<00:00, 2139.71it/s]


In [52]:
print('Len of index that met the condition: {}'.format(len(idx_met)))
print('Len of index that NOT met the condition: {}'.format(len(idx_nmet)))
print('Len of Null index : {}'.format(len(idx_null)))

Len of index that met the condition: 608781
Len of index that NOT met the condition: 127
Len of Null index : 129040


In [53]:
#Droping index that not met
df.drop(idx_nmet,inplace=True)
print(df.shape)
df.head()

(737821, 21)


,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02,11001,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,19,F,IMPORTADO,LEVE,ITALIA,2020-02-27,NaT,2020-03-06,2020-03-13,2020-03-06,PCR,11,380.0,OTRO,NO INFO
1,2,2020-03-06,76111,GUADALAJARA DE BUGA,VALLE DEL CAUCA,RECUPERADO,34,M,IMPORTADO,LEVE,ESPANA,2020-03-04,NaT,2020-03-09,2020-03-19,2020-03-09,PCR,76,724.0,NEGRO,NO INFO
2,3,2020-03-07,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,50,F,IMPORTADO,LEVE,ESPANA,2020-02-29,NaT,2020-03-09,2020-03-15,2020-03-09,PCR,5,724.0,OTRO,NO INFO
3,4,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,55,M,RELACIONADO,LEVE,NO INFO,2020-03-06,NaT,2020-03-11,2020-03-26,2020-03-11,PCR,5,NaN,OTRO,NO INFO
4,5,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,25,M,RELACIONADO,LEVE,NO INFO,2020-03-08,NaT,2020-03-11,2020-03-23,2020-03-11,PCR,5,NaN,OTRO,NO INFO


Reviewing the people who were diagnosed with COVID after death:

* 'Fecha diagnostico'<'Fecha de muerte'

In [61]:
idx_met=[]
idx_nmet=[]
idx_null=[]
for x in tqdm(df.index):
    if pd.notna(df['FECHA DIAGNOSTICO'][x]) and pd.notna(df['FECHA DE MUERTE'][x]):
        if df['FECHA DIAGNOSTICO'][x]<df['FECHA DE MUERTE'][x]:
            idx_met.append(x)
        else:
            idx_nmet.append(x)
    else:
        idx_null.append(x)

100%|██████████| 737821/737821 [11:21<00:00, 1082.59it/s]


In [62]:
print('{} People diagnosed with COVID after death'.format(len(idx_nmet)))

10668 People diagnosed with COVID after death


We can not drop those index as that is a insight of the People who were diagnosed with COVID after death

Checking that we have no values for both 'Fecha recuperado' and 'Fecha de muerte'

In [63]:
#filtering only not null values for Fecha recueprado in the df
print('Null values in DF for Fecha recuperado : \t')
print(df[pd.notnull(df['FECHA RECUPERADO'])]['FECHA RECUPERADO'].isnull().unique())


Null values in DF for Fecha recuperado : 	
[False]


In [64]:
#asking if there is a null value in each row, True is yes and False if there is a date
df[pd.notnull(df['FECHA RECUPERADO'])]['FECHA DE MUERTE'].isnull().unique()

array([ True, False])

We have dates in the same row for 'Fecha de mmuerte' and 'Fecha recueprado'.

We must drop those values

In [65]:
# #NO SE PEUDE HACER PORQUE NO ESOTY TENIENDO EN CUENTA OTROS VALARES NULOS, ESTARIA PERDIENDO INFORMACIÓN
# #checking the values that have no Fecha de muerte
# temp=df[pd.notnull(df['Fecha recuperado'])]
# idx_frfd=temp[temp['Fecha de muerte'].isnull()].index
# print(temp[temp['Fecha de muerte'].isnull()]['Fecha de muerte'].unique())
# idx_frfd

In [66]:
idx_met=[]
for x in df.index:
    if pd.notna(df['FECHA RECUPERADO'][x]) and pd.notna(df['FECHA DE MUERTE'][x]):
        idx_met.append(x)

In [67]:
print('Len of index that met the condition: {}'.format(len(idx_met)))

Len of index that met the condition: 287


We need to drop those values, because qe can not have a date for a recoverd people but also shows a date of death

In [68]:
#droping the values that present in both Fecha recuperado and Fecha de muerte 
df.drop(idx_met,inplace=True)
print(df.shape)
df.head()

(737534, 21)


,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02,11001,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,19,F,IMPORTADO,LEVE,ITALIA,2020-02-27,NaT,2020-03-06,2020-03-13,2020-03-06,PCR,11,380.0,OTRO,NO INFO
1,2,2020-03-06,76111,GUADALAJARA DE BUGA,VALLE DEL CAUCA,RECUPERADO,34,M,IMPORTADO,LEVE,ESPANA,2020-03-04,NaT,2020-03-09,2020-03-19,2020-03-09,PCR,76,724.0,NEGRO,NO INFO
2,3,2020-03-07,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,50,F,IMPORTADO,LEVE,ESPANA,2020-02-29,NaT,2020-03-09,2020-03-15,2020-03-09,PCR,5,724.0,OTRO,NO INFO
3,4,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,55,M,RELACIONADO,LEVE,NO INFO,2020-03-06,NaT,2020-03-11,2020-03-26,2020-03-11,PCR,5,NaN,OTRO,NO INFO
4,5,2020-03-09,5001,MEDELLIN,ANTIOQUIA,RECUPERADO,25,M,RELACIONADO,LEVE,NO INFO,2020-03-08,NaT,2020-03-11,2020-03-23,2020-03-11,PCR,5,NaN,OTRO,NO INFO


-----------------

### Cheking numerical data

In [69]:
df.select_dtypes(include=np.number).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737534 entries, 0 to 743944
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID DE CASO           737534 non-null  int64  
 1   CÓDIGO DIVIPOLA      737534 non-null  int64  
 2   EDAD                 737534 non-null  int64  
 3   CODIGO DEPARTAMENTO  737534 non-null  int64  
 4   CODIGO PAIS          961 non-null     float64
dtypes: float64(1), int64(4)
memory usage: 33.8 MB


We only have missing values in Codigo pais

In [72]:
#cheking if we have no duplicates in ID de caso
df['ID DE CASO'].value_counts().unique()

array([1])

In [74]:
df['ID DE CASO'].nunique()

737534

In [71]:
#df['Código DIVIPOLA'].unique()

In [75]:
#cheking unique values in Edad
print(sorted(df['EDAD'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 112, 115, 123]


It is worth noting look the people over 100 years old

In [76]:
#cheking unique values in Codigo departamento
print(sorted(df['CODIGO DEPARTAMENTO'].unique()))

[5, 8, 11, 13, 15, 17, 18, 19, 20, 23, 25, 27, 41, 44, 47, 50, 52, 54, 63, 66, 68, 70, 73, 76, 81, 85, 86, 88, 91, 94, 95, 97, 99]


In [78]:
#cheking unique values in Codigo pais
print(sorted(df['CODIGO PAIS'].unique()))

[56.0, 380.0, 724.0, nan, 32.0, 36.0, 50.0, 70.0, 76.0, 124.0, 152.0, 188.0, 191.0, 192.0, 214.0, 218.0, 250.0, 276.0, 300.0, 320.0, 340.0, 372.0, 376.0, 388.0, 404.0, 484.0, 504.0, 530.0, 531.0, 533.0, 558.0, 591.0, 604.0, 620.0, 630.0, 682.0, 752.0, 756.0, 760.0, 784.0, 792.0, 818.0, 826.0, 840.0, 858.0, 862.0]


In [79]:
#Checking the ages over 99 years old
df[df['EDAD']>99]#.select_dtypes(exclude='datetime')

,ID DE CASO,FECHA DE NOTIFICACIÓN,CÓDIGO DIVIPOLA,CIUDAD DE UBICACIÓN,DEPARTAMENTO O DISTRITO,ATENCIÓN,EDAD,SEXO,TIPO,ESTADO,PAÍS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACIÓN,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
3757,3758,2020-04-12,11001,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,103,F,EN ESTUDIO,LEVE,NO INFO,2020-04-11,NaT,2020-04-19,2020-04-24,2020-04-19,PCR,11,NaN,OTRO,NO INFO
8246,8287,2020-05-02,8001,BARRANQUILLA,BARRANQUILLA D.E.,FALLECIDO,100,F,EN ESTUDIO,FALLECIDO,NO INFO,2020-05-01,2020-05-01,2020-05-05,NaT,2020-05-05,NO INFO,8,NaN,OTRO,NO INFO
26612,26653,2020-05-26,11001,BOGOTA D.C.,BOGOTA D.C.,FALLECIDO,100,F,EN ESTUDIO,FALLECIDO,NO INFO,2020-05-26,2020-05-30,2020-05-29,NaT,2020-05-29,NO INFO,11,NaN,OTRO,NO INFO
28881,28922,2020-05-20,13001,CARTAGENA DE INDIAS,CARTAGENA D.T. Y C.,FALLECIDO,103,F,EN ESTUDIO,FALLECIDO,NO INFO,2020-05-16,2020-05-28,2020-05-31,NaT,2020-05-31,NO INFO,13,NaN,OTRO,NO INFO
38660,38701,2020-05-28,76364,JAMUNDI,VALLE DEL CAUCA,FALLECIDO,102,F,EN ESTUDIO,FALLECIDO,NO INFO,2020-05-25,2020-05-28,2020-06-07,NaT,2020-06-07,NO INFO,76,NaN,NEGRO,NO INFO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731371,731412,2020-09-04,5001,MEDELLIN,ANTIOQUIA,CASA,123,M,EN ESTUDIO,LEVE,NO INFO,2020-08-31,NaT,2020-09-15,NaT,2020-09-16,NO INFO,5,NaN,NO INFO,NO INFO
734461,734502,2020-09-04,5001,MEDELLIN,ANTIOQUIA,FALLECIDO,100,F,EN ESTUDIO,FALLECIDO,NO INFO,2020-08-31,2020-09-12,2020-09-15,NaT,2020-09-16,NO INFO,5,NaN,NO INFO,NO INFO
736462,736503,2020-08-31,20001,VALLEDUPAR,CESAR,HOSPITAL,101,F,EN ESTUDIO,MODERADO,NO INFO,2020-08-26,NaT,2020-08-31,NaT,2020-09-17,NO INFO,20,NaN,NO INFO,NO INFO
737682,737723,2020-09-12,8001,BARRANQUILLA,BARRANQUILLA D.E.,CASA,101,F,EN ESTUDIO,LEVE,NO INFO,2020-09-12,NaT,2020-09-16,NaT,2020-09-17,NO INFO,8,NaN,NO INFO,NO INFO


In [88]:
print('There are {} people over 99 years old, and the maximum age is:'.format(len(df[df['EDAD']>99])),df['EDAD'].max())

There are 142 people over 99 years old, and the maximum age is: 123


In [90]:
df[df['EDAD']>99].to_excel('personas_mayores_99.xlsx')

Droping those people

In [91]:
df.drop(df[df['EDAD']>99].index,inplace=True)

Check that for Estado=ASINTOMATICO have no Fecha de muerte

In [93]:
temp=df[df['ESTADO']=='ASINTOMATICO']
temp2=temp[temp['FECHA DE MUERTE'].notnull()]
temp2['M<D']=df['FECHA DIAGNOSTICO']>df['FECHA DE MUERTE']

Since we have asymptomatic people with a date of death, we must review their date of diagnosis, because possibly these people died and after they died they were diagnosed with COVID, however they did not die from COVID

In [94]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == True
    return ['background-color: yellow' if v else '' for v in is_max]

In [97]:
temp2[['ESTADO','FECHA RECUPERADO','FECHA DIAGNOSTICO','FECHA DE MUERTE','M<D']].style.apply(highlight_max)

,ESTADO,FECHA RECUPERADO,FECHA DIAGNOSTICO,FECHA DE MUERTE,M<D
790,ASINTOMATICO,NaT,2020-03-30 00:00:00,2020-06-10 00:00:00,False
11106,ASINTOMATICO,NaT,2020-05-11 00:00:00,2020-05-16 00:00:00,False
40737,ASINTOMATICO,NaT,2020-06-09 00:00:00,2020-06-06 00:00:00,True
49158,ASINTOMATICO,NaT,2020-06-14 00:00:00,2020-06-19 00:00:00,False
63176,ASINTOMATICO,NaT,2020-06-19 00:00:00,2020-07-13 00:00:00,False
79972,ASINTOMATICO,NaT,2020-06-25 00:00:00,2020-07-18 00:00:00,False
94248,ASINTOMATICO,NaT,2020-06-28 00:00:00,2020-08-06 00:00:00,False
105698,ASINTOMATICO,NaT,2020-07-02 00:00:00,2020-07-15 00:00:00,False
113223,ASINTOMATICO,NaT,2020-07-04 00:00:00,2020-07-11 00:00:00,False
115869,ASINTOMATICO,NaT,2020-07-05 00:00:00,2020-07-13 00:00:00,False


In fact, the highlighted values shows the people that died before diagnosed with COVID

In [98]:
print('We have {} "ASINTOMATICO" people that have dates of Fecha de dianostico and fecha de muerte'.format(len(temp2)))

We have 64 "ASINTOMATICO" people that have dates of Fecha de dianostico and fecha de muerte


Last check,we must not have values for Tipo de recuperacion when there is a date Fecha de muerte

In [101]:
df[pd.notnull(df['FECHA DE MUERTE'])]['TIPO RECUPERACIÓN'].unique()

array(['NO INFO'], dtype=object)

* Division politica administrativa DIVIPOLA
* FECHA Inicio sintomas FIS

In [104]:
df.to_pickle('pre_cleaned_dataset.pkl')

---------------------------------

###  Vamos a ver los casos activos por cada municipio, para ello iimportamos la polacion que el DANE proporciona

In [ ]:
df2=pd.read_excel('Datos municipios 2020.xlsx')
df2.head()

In [ ]:
#droping duplicate columns
df2.drop(columns=['Departamento','Nombre_Departamento','Misión_Rural'],inplace=True)

Cambiamos el nombre de Municipio a cod DIVIPOLA y 2020 a Población

In [ ]:
df2.rename(columns={'Municipio':'Código DIVIPOLA',2020:'Población'},inplace=True)

##### hicimos un merge tipo left de los dos dataframes

In [ ]:
mergedf=pd.merge(df,df2,how='left',on='Código DIVIPOLA')
mergedf.head()

In [ ]:
mergedf.atención.unique()

In [ ]:
#looking the atention values different than Fallecido, Recuperado and nan, the resulting df (temp_m1) contains only active cases
temp_m1=mergedf[~mergedf['atención'].isin(['Fallecido','Recuperado','NO INFO'])]#np.nan])]
temp_m1.tail()

In [ ]:
#grouping active COVID cases
casos_activos=temp_m1.groupby('Código DIVIPOLA')['ID de caso'].count().reset_index(name='total_casos_activos')
print(type(casos_activos))
casos_activos

In [ ]:
#Merged casos_activos with df2
casos_activos_poblacion=casos_activos.merge(df2,how='left', on='Código DIVIPOLA').rename(columns={2020:'poblacion'})
casos_activos_poblacion

In [ ]:
casos_activos_poblacion[casos_activos_poblacion['Nombre_Municipio']=='Chía']

In [ ]:
#Casos activos millon de habitantes
casos_activos_poblacion['casosxmillon']=1000000*casos_activos_poblacion['total_casos_activos']/casos_activos_poblacion['Población'] 

#sorting by casos por millon (casosxmillon)
casos_activos_poblacion.sort_values(by=['casosxmillon'],ascending=False,inplace=True)

casos_activos_poblacion

In [ ]:
casos_activos_poblacion.to_excel('tabla.xlsx')

In [ ]:
#casos_activos_poblacion['casosxmillon'].sort_values(ascending=False).head(10).plot(kind='bar')
casos_activos_poblacion.sort_values(by='casosxmillon',ascending=False).head(10).plot(x='Nombre_Municipio',y='casosxmillon',kind='bar')

In [ ]:
casos_activos_poblacion.groupby('Clima')['casosxmillon'].sum().reset_index()

---------------------

### Geo locations of Colombia

In [ ]:
geo=pd.read_csv('geo_municipios.csv')
geo.head(3)

In [ ]:
#adding values to latitud 
geo.latitud=geo.nombre_tipo_ctro_pbdo

In [ ]:
#removing the index because we have dupplicated index
geo.reset_index(drop=True,inplace=True)

In [ ]:
geo[geo['codigo_departamento']==99773]

Notice we have different names for nombre_departamento and nombre_municipio, we're only interested on those who match

In [ ]:
# #looking for the index that match on 'nombre_departamento' and'nombre_municipio'
# geo_idx=[]
# for nombre in geo.index:
#     if geo['nombre_departamento'][nombre]==geo['nombre_municipio'][nombre]:
#         geo_idx.append(nombre)

print(geo[geo['nombre_centro_poblado']=='CABECERA MUNICIPAL'].isnull().any())
geo[geo['nombre_centro_poblado']=='CABECERA MUNICIPAL']

In [ ]:
geo[geo['nombre_centro_poblado']=='CABECERA MUNICIPAL']['codigo_departamento'].value_counts().unique()

In [ ]:
#Updating geo DataFrame
geo_idx=geo[geo['nombre_centro_poblado']=='CABECERA MUNICIPAL'].index
geo=geo.loc[geo_idx]

In [ ]:
#Renaming codigo_departamento to Código DIVIPOLA, for future merge with casos_activos_poblacion DataFrame
geo.rename(columns={'codigo_departamento':'Código DIVIPOLA'}, inplace=True)

In [ ]:
#We are keeping codigo_departamento(Código DIVIPOLA) nombre_departamento longitud and latitud
geo=geo[['Código DIVIPOLA','nombre_departamento', 'longitud', 'latitud']]
geo.head()

In [ ]:
geo.isnull().any()

In [ ]:
geo[geo['Código DIVIPOLA']==11001]

Merging geo with casos_activos_poblacion

In [ ]:
geo_casos_activos=casos_activos_poblacion.merge(geo,how='left',on='Código DIVIPOLA')
geo_casos_activos.head()

In [ ]:
geo_casos_activos.isnull().any()

In [ ]:
temp=geo_casos_activos[(geo_casos_activos['longitud'].isnull())|(geo_casos_activos['latitud'].isnull())]

temp

In [ ]:
print('We have {} values without longitud or latitud'.format(len(temp)))

In [ ]:
temp['Nombre_Municipio'].unique()
#Create a DataFrame that contains 

In [ ]:
geo_casos_activos['nombre_departamento']=geo_casos_activos['Nombre_Municipio'].progress_apply(lambda x: x.upper()+' COLOMBIA')

In [ ]:
#Testing adding latitude and logitude if there is a nan in those columns

#uncomment for check example
# print(geolocator.geocode("Barranquilla Colombia")[1])
# (geolocator.geocode("Barranquilla Colombia").latitude,geolocator.geocode("Bogotá Colombia").longitude)

# uncomment for look in the DataFrame
# geo_casos_activos.progress_apply(lambda x: (geolocator.geocode(x.nombre_departamento))[1] if (pd.isna(x.longitud))|(pd.isna(x.latitud)) else (x.longitud,x.latitud),axis=1)

geo_casos_activos['latitud2']=geo_casos_activos.progress_apply(lambda x: (geolocator.geocode(x.nombre_departamento)).latitude if (pd.isna(x.longitud)) else x.longitud, axis=1)
geo_casos_activos['longitud2']=geo_casos_activos.progress_apply(lambda x: (geolocator.geocode(x.nombre_departamento)).longitude if (pd.isna(x.latitud)) else x.latitud, axis=1)

In [ ]:
geo_casos_activos

In [ ]:
geo_casos_activos['Nombre_Municipio'].isnull().any()

Aternate aproaches to look for the longitude and latitude

In [ ]:
#geo_casos_activos['latitud2']=geo_casos_activos['nombre_departamento'].progress_apply(lambda x: geolocator.geocode(x).latitude if geolocator.geocode(x) != None else 'NF')
#geo_casos_activos['longitud2']=geo_casos_activos['nombre_departamento'].progress_apply(lambda x: geolocator.geocode(x).longitude if geolocator.geocode(x) != None else 'NF')

#geo_casos_activos[geo_casos_activos['latitud2']=='NF']

In [ ]:
# a=np.where((geo_casos_activos.longitud.isna())|(geo_casos_activos.latitud.isna()),geo_casos_activos.Nombre_Municipio,(geo_casos_activos.longitud,geo_casos_activos.latitud))
# pd.DataFrame(a)

---------------------------

### Ploting on map

In [ ]:
geo_casos_activos.plot(kind="scatter", x="latitud2", y="longitud2")

In [ ]:
# c is the attribute we'll map onto colors, s is the attribute we'll represent with circle size.
geo_casos_activos.plot(kind="scatter", x="latitud2", y="longitud2",
    s=geo_casos_activos['casosxmillon']/100, label="CasosxMillon",
    c='Población', cmap=plt.get_cmap("jet"),
    colorbar=True, alpha=0.4, figsize=(10,7),
)
plt.legend()
#save_fig("housing_prices_scatterplot")
plt.show()

In [ ]:
def generateBaseMap(default_location=[4.00,-72.00], default_zoom_start=6.4):
    base_map = folium.Map(location=default_location,tiles='cartodbdark_matter', control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
base_map=generateBaseMap()

In [ ]:
from folium.plugins import HeatMap

#HeatMap(data=df[['longitud', 'latitud', 'casosxmillon']].groupby(['longitud', 'latitud']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

HeatMap(data=geo_casos_activos[['latitud2','longitud2', 'casosxmillon']].values.tolist(),radius=8, max_zoom=13).add_to(base_map)

# for i in range(0,len(geo_casos_activos)):
#     HeatMap(
#         data=[geo_casos_activos.iloc[i]['longitud2'], geo_casos_activos.iloc[i]['latitud2']],
#         popup=geo_casos_activos.iloc[i]['Nombre_Municipio'],
#         radius=geo_casos_activos.iloc[i]['casosxmillon']).add_to(base_map)

In [ ]:
base_map

In [ ]:
geo_casos_activos.reset_index(drop=True,inplace=True)

In [ ]:
colombia = folium.Map(location=[4.00,-72.00],zoom_start=6.4)

for i in range(0,len(geo_casos_activos)):
    folium.Circle(
        location=[geo_casos_activos.iloc[i]['latitud2'], geo_casos_activos.iloc[i]['longitud2']],
        popup=(unidecode.unidecode(geo_casos_activos.iloc[i]['Nombre_Municipio']),
               int(geo_casos_activos.iloc[i]['casosxmillon']),
               unidecode.unidecode('Clima: '+geo_casos_activos.iloc[i]['Clima']),
               'Altitud: '+ str(geo_casos_activos.iloc[i]['Altitud'])+'m'),
        radius=geo_casos_activos.iloc[i]['casosxmillon'],
        color='crimson',
        fill=True,
        fill_color='crimson').add_to(colombia)

colombia.save(outfile='casos_activos.html')
colombia

In [ ]:
geo_casos_activos.iloc[[6,121,183,265,401,619,789]]

In [ ]:
#Example adding radius to heatmap using folium

# import random
# from folium import plugins

# data = [[41.895278,12.482222,2873494.0,20.243001,20414,7.104243],
#         [41.883850,12.333330,3916.0,0.835251,4,1.021450],
#         [41.854241,12.567000,22263.0,1.132390,35,1.572115],
#         [41.902147,12.590388,19505.0,0.839181,37,1.896950],
#         [41.994240,12.48520,16239.0,1.383981,25,1.539504]]

# dfdfdf = pd.DataFrame(columns=['latitude','longitude','population','radius','count','normalized'],data=data)

# middle_lat = dfdfdf['latitude'].median()
# middle_lon = dfdfdf['longitude'].median()
# m = folium.Map(location=[middle_lat, middle_lon],tiles = "Stamen Terrain",zoom_start=11)

# # convert to (n, 2) nd-array format for heatmap
# points = dfdfdf[['latitude', 'longitude', 'normalized']].dropna().values

# # plot heatmap
# #plugins.HeatMap(points, radius=15).add_to(m)

# pointArrays = np.split(points, len(points))
# radii = [5, 10, 15, 20, 25]

# for point, radius in zip(pointArrays, radii):
#     plugins.HeatMap(point, radius=radius).add_to(m)

In [ ]:
# mp = folium.Map(location=[middle_lat, middle_lon],tiles = "Stamen Terrain",zoom_start=11)
mp = folium.Map(location=[4.00,-72.00],zoom_start=6.4,tiles='cartodbdark_matter')

points = geo_casos_activos[['longitud','latitud','casosxmillon']].dropna().values

pointArrays = np.split(points, len(points))
radio=(geo_casos_activos['casosxmillon']/1000).tolist()

for point, radius in zip(pointArrays, radio):
    HeatMap(point,radius=radius).add_to(mp)
    
mp.save(outfile='casos_activos_heatmap.html')

In [ ]:
#print(tuple(zip(pointArrays, radio)))